In [112]:
import requests
import base64
from bs4 import BeautifulSoup
import pandas
import sqlite3 as lite
from shutil import copyfileobj
import smtplib
import mimetypes
from email.mime.multipart import MIMEMultipart
from email import encoders
from email.message import Message
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from email.mime.image import MIMEImage
from email.mime.text import MIMEText
from datetime import datetime
import sys


#get time of day 
currenttime = datetime.now()

#payload data for twse
payload={}
dt_str = []
#gmail setting
emailfrom = "gghammer@gmail.com"
emailto = "gghammer@gmail.com"
#fileToSend = "daily.xlsx"
username = "gghammer"
password = "ggpudin81161"



def send_to_gmail(source,attach_name):    
    #save to xlsx file
    writer = pandas.ExcelWriter(attach_name, engine='xlsxwriter')
    source.to_excel(writer, sheet_name='Sheet1')
    writer.save()

    #gmail send 
    msg = MIMEMultipart()
    msg["From"] = emailfrom
    msg["To"] = emailto
    msg["Subject"] = "哈墨投顧日報 "+currenttime.strftime('%Y-%m-%d')
    msg.preamble = "help I cannot send an attachment to save my life"

    ctype, encoding = mimetypes.guess_type(attach_name)
    if ctype is None or encoding is not None:
        ctype = "application/octet-stream"

    maintype, subtype = ctype.split("/", 1)

    if maintype == "text":
        fp = open(attach_name)
        # Note: we should handle calculating the charset
        attachment = MIMEText(fp.read(), _subtype=subtype)
        fp.close()
    elif maintype == "image":
        fp = open(attach_name, "rb")
        attachment = MIMEImage(fp.read(), _subtype=subtype)
        fp.close()
    elif maintype == "audio":
        fp = open(attach_name, "rb")
        attachment = MIMEAudio(fp.read(), _subtype=subtype)
        fp.close()
    else:
        fp = open(attach_name, "rb")
        attachment = MIMEBase(maintype, subtype)
        attachment.set_payload(fp.read())
        fp.close()
        encoders.encode_base64(attachment)
    attachment.add_header("Content-Disposition", "attachment", filename=attach_name)
    msg.attach(attachment)

    server = smtplib.SMTP("smtp.gmail.com:587")
    server.starttls()
    server.login(username,password)
    server.sendmail(emailfrom, emailto, msg.as_string())
    server.quit()


def twse_to_sql_指標(source,target):  
    #catch payload 
    res = requests.post('http://www.tse.com.tw/ch/trading/exchange/BWIBBU/BWIBBU_d.php#')
    res.encoding = 'big5'

    #find key & Encryption 
    soup = BeautifulSoup(res.text, 'html.parser') 

    #get & translate date 
    dt = soup.select('input')[2].get('value')
    dt = dt.replace(dt[0:3], str(int(dt[0:3])+ 1911))
    dt = dt.replace(dt[4:5],'-')
    dt_str = dt.split(' ')

    for inp in soup.select('input'):
        if 'hidden' in inp.get('type'): 
            payload[inp.get('name')] = base64.b64encode(inp.get('value').encode('utf-8')) 
        
    #download csw file        
    res2 = requests.post('http://www.tse.com.tw/ch/trading/exchange/BWIBBU/BWIBBU_print.php?language=ch&save=csv',data = payload ,stream = True)

    #save to twse.csv 
    f = open(source,'wb')
    copyfileobj (res2.raw,f)
    f.close()  

    #read csw to pandas 
    twse = pandas.read_csv(source,encoding = "big5",header=1)

    with lite.connect(target) as conn:   
        for i in range(1,893):
            col = twse.iloc[i]
            col = col.tolist()
            data = dt_str + col
            conn.execute('INSERT INTO 指標 (日期,代號,名稱,本益比,殖利率,股價淨值比) VALUES (?,?,?,?,?,?);', data)
    conn.close()
    
def csw_to_sql_股票代號(source,target):    

    #read csw to pandas 
    twse = pandas.read_csv(source,encoding = "big5",header=1)

    #update to sqlite file 
    with lite.connect(target) as conn:   
        for i in range(1,893):
            col = twse.iloc[i]
            col = col.tolist()
            del col[2:5]
            conn.execute('INSERT INTO 股票代號 (代號,名稱) VALUES (?,?);', col)  
    conn.close()   
    
def create_sql_lib(filename):    
    #create sqlite lib
    conn = lite.connect(filename)
    print ("Opened database successfully")

    #save to sqlite file 
    with lite.connect(filename) as conn:
    
        conn.execute('''CREATE TABLE 股票代號
                    (代號 VARCHAR(10),名稱 VARCHAR(10));''')    
    
        conn.execute('''CREATE TABLE 歷史股價
                    (日期 DATE,代號 VARCHAR(10),名稱 VARCHAR(10),開盤價 INTEGER,收盤價 INTEGER,最高 INTEGER,最低 INTEGER);''')   
        
        conn.execute('''CREATE TABLE 指標
                    (日期 DATE,代號 VARCHAR(10),名稱 VARCHAR(10),本益比 INTEGER,殖利率 INTEGER,股價淨值比 INTEGER);''')    
    
        conn.execute('''CREATE TABLE 營收
                    (日期 DATE,代號 VARCHAR(10),名稱 VARCHAR(10),EPS INTEGER,月營收 INTEGER,季營收 INTEGER,年營收 INTEGER,配息 INTEGER,配股 INTEGER);''')     

        conn.execute('''CREATE TABLE 技術面
                    (日期 DATE,代號 VARCHAR(10),名稱 VARCHAR(10),
                    日量 INTEGER,週量 INTEGER,月量 INTEGER,
                    五日K INTEGER,二十日K INTEGER,五週k INTEGER,二十週k INTEGER,
                    五日KD INTEGER,五週kD INTEGER,二十週kD INTEGER);''') 
    
    print ("Table created successfully")
    conn.close()    


#main   
def main(fucntion):

    if fucntion == 'crawl':      
        twse_to_sql_指標('twse.csv','twse.sqlite')  #source csv file name ,target sql file name
        csw_to_sql_股票代號('twse.csv','twse.sqlite') #source csv file name , target sql file name
    elif fucntion == 'create_lib':
        create_sql_lib('twse.sqlite')
    elif fucntion == 'report':
        twse = pandas.read_csv('twse.csv',encoding = "big5",header=1)
        send_to_gmail(twse,'daily.xlsx') #source pandas structure , target excel file name        
    else:
        print('usage:')
        print('      hamu crawl #爬當日資料')
        print('      hamu create_lib #建立資料庫')        
        
        
if __name__ == '__main__':
    main(sys.argv[1])        


[]
